In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [2]:
DRAFT_YEAR = 2025

# NFL DRAFT BUZZ
l = []
df = pd.DataFrame()
count = 0
while True:
    count += 1
    url = requests.get(f'https://www.nfldraftbuzz.com/positions/ALL/{count}/{DRAFT_YEAR}')
    parse = BeautifulSoup(url.text, "lxml").prettify()
    start = parse.find("<table")
    end = parse.find("</table")
    df = pd.read_html(parse[start:end+8])[0]

    if 0 in df.columns:
        break
    else:
        l.append(df)

NFLDB = pd.concat(l)
NFLDB = NFLDB.rename(columns = {'H/W/S': 'Pos', '#' : 'Rank'})
NFLDB['Player'] = NFLDB['Player'].apply(lambda x: ' '.join(x.split()[1:]))
NFLDB['Rating'] = NFLDB['Rating'].apply(lambda x: x.split()[0]).astype(float)
NFLDB['Weight'] = NFLDB['Weight'].apply(lambda x: x.split('lbs')[0]).astype(int)
NFLDB.reset_index(drop = True, inplace = True)
NFLDB.drop(['Pos', 'Team'], axis = 1, inplace = True)
NFLDB.insert(loc = 0, column = 'SOURCE', value = 'NFLDraftBuzz')

NFLDB.to_csv('Data/DraftPool.csv')
NFLDB

,SOURCE,Rank,Player,Weight,Height,40YD,AVG POS RANK,AVG OVR RANK,Rating,Summary
0,NFLDraftBuzz,1,Travis Hunter,185,6-1,4.40,1.0,1.7,93.4,The most intriguing debate surrounding Hunter ...
1,NFLDraftBuzz,2,Ashton Jeanty,215,5-9,4.42,1.0,6.0,92.5,Jeanty's film study reveals a generational tal...
2,NFLDraftBuzz,3,Abdul Carter,259,6-3,4.48,1.0,2.6,91.1,The team that drafts Carter will be getting an...
3,NFLDraftBuzz,4,Mason Graham,320,6-3,5.10,2.0,5.0,90.9,Graham fits the bill of an immediate impact st...
4,NFLDraftBuzz,5,Will Johnson,202,6-2,4.39,2.0,10.3,89.6,"The NFL cornerback prototype is changing, and ..."
...,...,...,...,...,...,...,...,...,...,...
497,NFLDraftBuzz,498,Deion Smith,200,6-0,4.50,47.0,499.0,55.0,"His vision is outstanding, and Smith has an in..."
498,NFLDraftBuzz,499,Zykeivous Walker,291,6-4,4.75,96.0,500.0,54.0,Walker has an impressive blend of length and m...
499,NFLDraftBuzz,500,Jermayne Lole,315,6-3,4.95,104.0,576.5,54.0,"Despite a bowling-ball build, Lole has good fe..."
500,NFLDraftBuzz,501,Justin Wright,240,6-3,4.80,63.0,581.0,54.0,"Quick to read run, showing good agility and sp..."


In [ ]:
# Tankathon Full Order

def team_trade(x):
    if x[-2:].isupper():
        x = x.split()
        x.pop(0)
        x.pop(-1)
        x = ' '.join(x)
        return x
    else:
        return x
    
def team_double(x):
    # print(x[:(len(x) // 2)].strip())
    # print(x[(len(x) // 2):].strip())
    if x[:(len(x) // 2)].strip() == x[(len(x) // 2):].strip():
        return x[:(len(x) // 2)].strip()
    else:
        return x

l = []
url = requests.get(f'https://www.tankathon.com/nfl/full_draft')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = [m.start() for m in re.finditer('<table', parse)]
end = [m.start() for m in re.finditer('</table', parse)]

for i in range(0, len(start)):
    round = pd.DataFrame()
    round = pd.read_html(parse[start[i]:end[i]+8])[0]
    round.insert(loc = 0, column = 'Round', value = int(i) + 1)
    round.insert(loc = 0, column = 'SOURCE', value = 'Tankathon')
    l.append(round)

DRAFT_ORDER = pd.concat(l)
DRAFT_ORDER.reset_index(drop = True, inplace = True)
DRAFT_ORDER.columns = ['SOURCE', 'Round', 'Pick', 'Team']
DRAFT_ORDER['Pick'] = DRAFT_ORDER['Pick'].astype('Int64')
DRAFT_ORDER = DRAFT_ORDER[~DRAFT_ORDER['Team'].str.contains('Forfeited')]
DRAFT_ORDER['Team'] = DRAFT_ORDER['Team'].apply(team_trade)
DRAFT_ORDER['Team'] = DRAFT_ORDER['Team'].apply(team_double)
DRAFT_ORDER.to_csv('Data/DraftOrder.csv')
DRAFT_ORDER

,SOURCE,Round,Pick,Team
0,Tankathon,1,1,Tennessee
1,Tankathon,1,2,Cleveland
2,Tankathon,1,3,NY Giants
3,Tankathon,1,4,New England
4,Tankathon,1,5,Jacksonville
...,...,...,...,...
254,Tankathon,7,253,Miami
255,Tankathon,7,254,New Orleans
256,Tankathon,7,255,Cleveland
257,Tankathon,7,256,LA Chargers
